In [10]:
import dipy.reconst.dti as dti
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np

from dipy.core.gradients import gradient_table
from dipy.data import get_sphere
from dipy.io import read_bvals_bvecs
from dipy.io.image import save_nifti
from dipy.reconst.dti import fractional_anisotropy, color_fa, lower_triangular
from dipy.reconst.dti import TensorModel
from dipy.viz import window, actor


In [4]:
# Import DWI data, bvec, bval, and binary mask
fimg = ("/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/1000/from_mgh/data.nii.gz")
img = nib.load(fimg)
data = img.get_data()
print('data.shape (%d, %d, %d, %d)' % data.shape)
affine = img.affine #no need to store the affine tho. 

#Bval and bvec file information import
fbval=("/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/1000/from_mgh/bvals")
fbvec = ("/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/1000/from_mgh/bvecs")
bvals, bvecs = read_bvals_bvecs(fbval, fbvec)
gtab = gradient_table(bvals,bvecs)
#print(gtab.bvals) #to read the bvals


#Read the voxel size from the image header.
voxel_size = img.header.get_zooms()[:3]

#To import fod?
#sh_data = nib.load(args.sh_file).get_data().astype('float64')

#importing binary mask
mask=("/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/1000/mask_con.nii.gz")
img = nib.load(mask)
mask = img.get_data()
print('mask.shape (%d, %d, %d)' % mask.shape)

data.shape (140, 140, 96, 69)
mask.shape (140, 140, 96)


In [5]:
#Fit the tensor model to data
tenmodel = dti.TensorModel(gtab)
tenfit = tenmodel.fit(data, mask)

#Compute dti maps
fa = fractional_anisotropy(tenfit.evals)
# In the background of the image the fitting will not be accurate there is no signal and possibly we will find FA values
# with nans (not a number). We can easily remove these in the following way.
# FA[np.isnan(FA)] = 0
# fa_img = nib.Nifti1Image(FA.astype(np.float32), img.affine)
# nib.save(fa_img, 'tensor_fa.nii.gz')

# #We can also compute the colored FA or RGB-map [Pajevic1999]. First, we make sure that the FA is scaled between 0 and 1,
# we compute the RGB map and save it.
# FA = np.clip(FA, 0, 1)
# RGB = color_fa(FA, tenfit.evecs)
# nib.save(nib.Nifti1Image(np.array(255 * RGB, 'uint8'), img.affine), 'tensor_rgb.nii.gz')

Save Tensors 

In [13]:
import numpy as np 
FA = np.clip(fa, 0, 1)
RGB = color_fa(FA, tenfit.evecs)
nib.save(nib.Nifti1Image(np.array(255 * RGB, 'uint8'), img.affine), '/space/hemera/1/users/cmaffei/data_30_05/rgb.nii.gz')

In [ ]:
#Visualization

sphere = get_sphere('symmetric724')

# Enables/disables interactive visualization
interactive = True
ren = window.Renderer()

evals = tenfit.evals[63:64, 57:87, 43:73]
evecs = tenfit.evecs[63:64, 57:87, 43:73]

# We can color the ellipsoids using the color_fa values that we calculated above.
# In this example we additionally normalize the values to increase the contrast.
cfa = RGB[13:43, 44:74, 28:29]
cfa /= cfa.max()

# We can visualize the tensor Orientation Distribution Functions for the same area as we did with the ellipsoids.
# We first mask the data
# maskdata_cut, mask = median_otsu(data_cut, 3, 1, True,
#                                  vol_idx=range(10, 50), dilate=2)
# 
#Axial
# index = data.shape[2] // 2
# data_cut_z = data [:, :, index:index+1]
# tensor_odfs = tenmodel.fit(data_cut_z).odf(sphere)
# odf_actor = actor.odf_slicer(tensor_odfs, sphere=sphere, scale=0.5)
# odf_actor.display(z=0)
# ren.set_camera(position=(0, 0, 10), view_up=(0, 1, 0))
# ren.add(odf_actor)
# print('Saving illustration as tensor_odfs.png')
# window.record(ren, n_frames=1, out_path='tensor_odfs_axial.png', size=(1024, 800))
# if interactive:
#     window.show(ren)

# Sagittal View
# index = data.shape[0] // 3
# data_cut_y = data [index:index+1, :, :]
# tensor_odfs = tenmodel.fit(data_cut_y).odf(sphere)
# odf_actor = actor.odf_slicer(tensor_odfs, sphere=sphere, scale=0.5, colormap='Reds')
# odf_actor.display(x=0)
# ren.set_camera(position=(0, 0, 10), view_up=(0, 1, 0))
# ren.add(odf_actor)
# print('Saving illustration as tensor_odfs.png')
# window.record(ren, n_frames=1, out_path='tensor_odfs_sagittal.png', size=(1024, 800))
# if interactive:
#     window.show(ren)

# Coronal View
# index = data.shape[1] // 2
# data_cut_x = data [:, index:index+1, :]
# tensor_odfs = tenmodel.fit(data_cut_x).odf(sphere)
# odf_actor = actor.odf_slicer(tensor_odfs, sphere=sphere, scale=0.5)
# odf_actor.display(y=0)
# ren.set_camera(position=(0, 0, 10), view_up=(0, 1, 0))
# ren.add(odf_actor)
# print('Saving illustration as tensor_odfs.png')
# window.record(ren, n_frames=1, out_path='tensor_odfs_coronal.png', size=(1024, 800))
# if interactive:
#     window.show(ren)

#You may wonder how we knew how to set the camera. This is very easy. You just need to run window.show once 
#see how you want to see the object and then close the window and call the camera_info method which prints the position,
#focal point and view up vectors of the camera. 
ren.camera_info()